### Question Three

##### Part One

We are asked to optimize a way of building a stadium. This is very similar to the example we did in class. We are given a list of tasks which need to be completed (some depend on previous ones). Ultimately, we are asked with finding the duration of achieving such a feat under the guidance of our given values. 

In [17]:
# this array stores the task number (1, 2, ..... 18)
tasks = []
for i = 1:18
    push!(tasks, i)    # push task number into array
end

# this dictionary stores the project durations
dur = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]
duration = Dict(zip(tasks,dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ([], [:1], [:2], [:2], [:3], [:4, :5], [:4], [:6], [:4, :6], [:4], [:6], [:9], [:7], [:2], [:4, :14],
            [:8, :11, :14], [:12], [:17])
pred = Dict(zip(tasks,pre));

In [18]:
using JuMP,Clp
m = Model(solver=ClpSolver())

@variable(m, tstart[tasks])

for i in tasks
    for j in pred[i]
        @constraint(m, tstart[i] >= tstart[j] + duration[j])
    end
end
@constraint(m, tstart[:1] == 0)
@objective(m, Min, tstart[18] + duration[18])     # total duration is start time of last task + duration of last task.

solve(m)
println(getvalue(tstart))
println("minimum duration: ", getobjectivevalue(m))
status = solve(m)
println(status)

tstart: 1 dimensions:
[ 1] = -0.0
[ 2] = 2.0
[ 3] = 18.0
[ 4] = 18.0
[ 5] = 27.0
[ 6] = 37.0
[ 7] = 26.0
[ 8] = 43.0
[ 9] = 43.0
[10] = 26.0
[11] = 43.0
[12] = 52.0
[13] = 28.0
[14] = 18.0
[15] = 26.0
[16] = 46.0
[17] = 54.0
[18] = 63.0
minimum duration: 64.0
Optimal


##### Part Two

We are asked to optimize a way of building a stadium again, but this time we are looking to find a way to see if we can make it occur in as few weeks as possible. This is to maximize the incentive pay out ($30k/week). 

In [1]:
using JuMP, Clp

m = Model(solver=ClpSolver())

# this array stores the task names (1, 2, ..., 18)
jobs = []
for i = 1:18
    push!(jobs, i)
end

# project durations
dur = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]
duration = Dict(zip(jobs,dur))

# projects based on ancestors
pre = ([], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17])
pred = Dict(zip(jobs,pre));

maxReduc =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0]  # max reduction (weeks)
costReduc = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]  # cost of reduction ($1,000/week)
bonus = 30                                                     # bonus for expediting the project ($1,000/week )

# variables 
@variable(m, tstart[jobs])
@variable(m, daysReduc[jobs] >= 0)
@variable(m, cost[jobs] >= 0)

# constraints 
for i in jobs # i is one day after j
    for j in pred[i]
        @constraint(m, daysReduc[j] <= maxReduc[j])
        @constraint(m, tstart[i] >= tstart[j] + duration[j] - daysReduc[j])
        @constraint(m, cost[j] >= daysReduc[j]*costReduc[j])
    end
end

@constraint(m, tstart[1] == 0)

# objective goal
@objective(m, Max, bonus*(66-(tstart[18] + duration[18]))-sum(cost))

solve(m)
println(getvalue(tstart))
println("Completed at: ",getvalue(tstart[18]+1))
println("Maximum profit: ", getobjectivevalue(m))

tstart: 1 dimensions:
[ 1] = -0.0
[ 2] = 2.0
[ 3] = 15.0
[ 4] = 15.0
[ 5] = 23.0
[ 6] = 31.0
[ 7] = 23.0
[ 8] = 36.0
[ 9] = 36.0
[10] = 23.0
[11] = 36.0
[12] = 45.0
[13] = 25.0
[14] = 15.0
[15] = 23.0
[16] = 39.0
[17] = 47.0
[18] = 53.0
Completed at: 54.0
Maximum profit: 147.0
